import required packages

In [2]:
# Third party packages.
import pandas as pd             # Data handling
import numpy as np              # Numeric calculations
import pickle                   # Save and load data
import altair as alt            # Visualize data
# import seaborn as sns           # Visualize data
import matplotlib.pyplot as plt # Visualize data

from scipy.stats import pearsonr # Correlation

load pkl file

In [3]:
data_path = r'C:\Users\Sandeep\Documents\Work\code\candle-pattern-test\data\nifty\samples\tradebook-ZH5601-FO.csv'

df = pd.read_csv(data_path)

df_ned_json_data = df

set pandas dataframe settings

In [4]:
# Setting Pandas options.
pd.set_option("display.max_rows", 50) # How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 100)
pd.set_option("display.max_info_rows", 1000000)
pd.set_option("display.precision", 2)
#pd.set_option("styler.format.precision", 2)

Descriptive analytics

In [5]:
# Show a sample of first 3 rows.
df_ned_json_data.head(3)

,symbol,isin,trade_date,exchange,segment,series,trade_type,auction,quantity,price,trade_id,order_id,order_execution_time,expiry_date
0,NIFTY2590224550PE,NaN,2025-09-01,NSE,FO,NaN,buy,False,75.0,50.65,1415959,1100000029001342,2025-09-01T09:53:54,2025-09-02
1,NIFTY2590924550CE,NaN,2025-09-01,NSE,FO,NaN,buy,False,75.0,185.70,1448983,1100000029699429,2025-09-01T09:54:54,2025-09-09
2,NIFTY2590924600CE,NaN,2025-09-01,NSE,FO,NaN,sell,False,75.0,157.40,1181069,1200000028157463,2025-09-01T09:54:58,2025-09-09


Data Understanding

a. Which variables are numerical and which can categorical? How many variables are of both types?

In [6]:
print(
    f"Number of columns in the original data: "
    f"{df_ned_json_data.shape[1]}"
    "\n"
)

df_orig_num    = df_ned_json_data.select_dtypes(include='number')
l_df_num_names = df_orig_num.columns.tolist()

print(f"\nNumber of numerical variables: {len(l_df_num_names)}")
print(l_df_num_names)


df_orig_cat    = df_ned_json_data.select_dtypes(include='object')
l_df_cat_names = list(df_orig_cat.columns)

print(f"\nNumber of categorical variables: {len(l_df_cat_names)}")
print(l_df_cat_names)


Number of columns in the original data: 14


Number of numerical variables: 6
['isin', 'series', 'quantity', 'price', 'trade_id', 'order_id']

Number of categorical variables: 7
['symbol', 'trade_date', 'exchange', 'segment', 'trade_type', 'order_execution_time', 'expiry_date']


Next to check for missing values, variable completeness

df_orig_num.info()

Create a frequency table counting number of missing values per variable

In [7]:
# Pandas Series with type of each variable (variable, column) in df_orig.
ps_missing_type    = df_ned_json_data.dtypes

# Number of missing data per variable.
ps_missing_total   = df_ned_json_data.isnull().sum()

# Percentage of missing per variable.
ps_missing_percent = round(100 * ps_missing_total / df_ned_json_data.shape[0], 1)

In [8]:
# Create table (Pandas DataFrame).
df_missing_data = pd.DataFrame({

    'data_type':   ps_missing_type,
    'empty_total': ps_missing_total,
    'empty_perc':  ps_missing_percent
})

# Sort table by number of missing data in descending order.
df_missing_data.sort_values(
    by        = 'empty_total',
    ascending = False,
    inplace   = True
)

# Remove variables that have no missing values.
df_missing_data = df_missing_data.query("empty_total > 0")

# Show table.
print(
    f"Number of variables having missing data: "
    f"{df_missing_data.shape[0]} (out of {df_ned_json_data.shape[1]})"
)

df_missing_data

Number of variables having missing data: 2 (out of 14)


,data_type,empty_total,empty_perc
isin,float64,739,100.0
series,float64,739,100.0


Conduct descriptive / summary statistics

In [9]:
df_orig_num.describe()
df_orig_cat.describe()

,symbol,trade_date,exchange,segment,trade_type,order_execution_time,expiry_date
count,739,739,739,739,739,739,739
unique,206,27,1,1,2,615,9
top,BANKNIFTY25NOV56500CE,2025-09-26,NSE,FO,buy,2025-09-26T11:09:30,2025-10-28
freq,13,98,739,739,371,10,221


data preparation phase

In [10]:
df_orig_num.median()

isin             NaN
series           NaN
quantity    7.50e+01
price       2.41e+02
trade_id    5.44e+05
order_id    1.60e+15
dtype: float64

Impute missing values

In [11]:
df_imputed_num = df_orig_num.replace(np.nan, df_orig_num.median())

print("Median values in original numerical data (first five variables):")
print(df_orig_num.median().head(5))

print("\nMedian values in imputed numerical data (first five variables):")
print(df_imputed_num.median().head(5))


print(
    f"\nNumber of missing data in imputed data: "
    f"{df_imputed_num.isna().sum().sum()}"
)

Median values in original numerical data (first five variables):
isin              NaN
series            NaN
quantity        75.00
price          241.05
trade_id    543536.00
dtype: float64

Median values in imputed numerical data (first five variables):
isin              NaN
series            NaN
quantity        75.00
price          241.05
trade_id    543536.00
dtype: float64

Number of missing data in imputed data: 1478


Imput missing values for categorical variable

In [12]:
df_orig_cat.mode()

,symbol,trade_date,exchange,segment,trade_type,order_execution_time,expiry_date
0,BANKNIFTY25NOV56500CE,2025-09-26,NSE,FO,buy,2025-09-26T11:09:30,2025-10-28
